# 1- Récupérer et faire un premier nettoyage des données

Dans ce notebook, on va collecter toutes les parties que j'ai jouées sur Lichess et les transformer en un jeu de données nettoyé pour construire un modèle de prédiction de mes défaites.

**Récupérer les données sur l'API Lichess**

On va utiliser l'API de Lichess pour télécharger toutes les parties que j'ai jouées sur le site.                 
Il suffit d'effectuer la requête: https://lichess.org/api/games/user/{username}                     
et de remplacer {username} par le nom de l'utilisateur qui nous intéresse.                      
Dan mon cas, il s'agit de Younessovitch.

**Importer les librairies**

In [11]:
import pandas as pd #données tabulaires
import numpy as np #algébre linéaire

**Transformer le fichier pgn en pandas dataframe**

In [12]:
def parse_pgn(file_path):
    with open(file_path, 'r') as file:
        pgn_data = file.read()
    games = pgn_data.strip().split('\n\n\n')
    game_details = []
    for game in games:
        details = {}
        lines = game.split('\n')
        for line in lines:
            if line.startswith('['):
                key, value = line.strip('[]').split(' ', 1)
                details[key] = value.strip('"')
            else:
                details['Moves'] = line.strip()
        game_details.append(details)
    return pd.DataFrame(game_details)

filename = 'lichess_pgn.pgn' #le fichier contenant toutes les parties
df_games = parse_pgn(filename)

In [13]:
df_games.iloc[0]

Event                                               Rated Blitz game
Site                                    https://lichess.org/BWrCDI0J
Date                                                      2023.12.16
White                                                  Younessovitch
Black                                                       llewor32
Result                                                           1-0
UTCDate                                                   2023.12.16
UTCTime                                                     08:31:29
WhiteElo                                                        1495
BlackElo                                                        1526
WhiteRatingDiff                                                   +6
BlackRatingDiff                                                   -6
Variant                                                     Standard
TimeControl                                                    180+2
ECO                               

A ce stade, nos données sont transformées en pandas dataframe.

On va étudier chaque caractéristique une par une pour en extraire l'information utile

**Event**

On va se séparer de cette caractéristique car TimeControl offre des informaitons plus précises sur les temps de parties jouées

**Site**

La caractéristique Site est magnifique pour une analyse humaine des parties. Elle ne se prêtera en revanche pas à notre étude. On va s'en séparer.

**Date**

On va se séparer de cette caractéristique car elle fait doublon avec UTCDate

**White and Black**                 
On va extraire la couleur avec laquelle j'ai joué et le nom de mon adversaire

**Result**

On va extraire mon résultat

**UTCDate et UTCTime**               
On va extraire la date et l'heure précise de la partie

**WhiteElo et BlackElo**

On va extraire mon Elo et celui de mon adversaire ainsi que la différence entre les deux 

**WhiteRatingDiff et BlackRatingDiff**

On va extraire l'évolution de mon classement et celle de mon adversaire

**Variant**                 
On va se séparer de cette caractéristique car elle ne prend qu'une seule valeur

**TimeControl**

In [14]:
df_games['TimeControl'].value_counts()

180+2      10362
60+0        7687
600+0        132
300+0         54
120+1         46
-             36
300+3         32
10800+0        2
180+0          1
300+4          1
Name: TimeControl, dtype: int64

On ne garde que les formats 180+2s et 60+0s car ils représentent la quasi totalité des parties

**ECO**

La caractéristique ECO permet de caractériser la variante jouée dans la partie (l'ouverture en gros). On ne va pas y toucher. C'est une caractéristique super importante!

In [15]:
df_games['ECO'].value_counts().describe()

count     243.000000
mean       75.526749
std       159.874209
min         1.000000
25%         2.500000
50%        13.000000
75%        71.000000
max      1214.000000
Name: ECO, dtype: float64

**Termination**                 
On va utiliser Termination et Moves pour construire le mode de terminaison d'une partie (abandon, montre, échec et mat)

**Moves**               
On va extraire le nombre de coups de la partie. On va aussi garder Moves tel quel. Il y a de la valeur dans cette caractéristique. On verra dans la suite comment l'exploiter.

In [40]:
def transform_data(df_games=df_games):
    df_games = df_games.copy()
    
    #remove rows with missing entries
    df_games = df_games.dropna()
    #transformer la date en datetime
    df_games['DateTime'] = pd.to_datetime(df_games['UTCDate'] +' '+ df_games['UTCTime'])
    
    #ma couleur
    my_name = 'Younessovitch'
    df_games['MyColor'] = df_games['White'].apply(lambda x: 'White' if (x == my_name) else 'Black')
    
    #nom de l'adversaire
    df_games['OpponentName']= df_games.apply(lambda x: x.Black if x.MyColor=='White' else x.White, axis=1)

    #mon résultat (tout ce qui n'est pas une victoire est une défaite)
    def get_result(color,result):
        return (color=='White' and result=='1-0') or (color=='Black' and result=='0-1')

    df_games['MyResult'] = df_games.apply(lambda x: 'Win' if ((x.MyColor=='White' and x.Result=='1-0') or (x.MyColor=='Black' and x.Result=='0-1')) else 'Defeat', axis=1)
    df_games['Win']=(df_games['MyResult']=='Win')
    #Mon Elo et celui de mon adversaire
    df_games['WhiteElo'] = df_games['WhiteElo'].replace({'?':'0'})
    df_games['WhiteElo'] = df_games["WhiteElo"].astype(int)
    df_games['BlackElo'] = df_games['BlackElo'].replace({'?':'0'})
    df_games['BlackElo'] = df_games["BlackElo"].astype(int)
    def get_elo(color, white_elo, black_elo):
        if color == 'White':
            return white_elo
        else:
            return black_elo
    #Mon Elo
    df_games['MyElo'] = df_games.apply(lambda x: get_elo(x.MyColor,x.WhiteElo,x.BlackElo),axis=1)
    
    #Elo de mon adversaire
    df_games['OpponentElo'] = df_games.apply(lambda x: get_elo(x.MyColor,x.BlackElo,x.WhiteElo),axis=1)
    
    #Différence d'Elo au début de la partie
    df_games['EloDifference'] = df_games['MyElo']-df_games['OpponentElo']
    
    #Evolution des classements à la fin de la partie
    df_games['WhiteRatingDiff'] = df_games['WhiteRatingDiff'].dropna()
    df_games['BlackRatingDiff'] = df_games['BlackRatingDiff'].dropna()    
    df_games['MyRatingDiff']= df_games.apply(lambda x: int(x.WhiteRatingDiff) if x.MyColor=='White' else int(x.BlackRatingDiff), axis=1)
    df_games['OpponentRatingDiff']= df_games.apply(lambda x: int(x.WhiteRatingDiff) if x.MyColor=='Black' else int(x.BlackRatingDiff), axis=1)

    #Mode de terminaison de la partie
    def get_finish(termination, moves):
        if termination == 'Time forfeit':
            return 'TimeOut'
        elif '#' in moves:
            return 'Checkmate'
        else:
            return 'Resignment'
    df_games['Finish'] = df_games.apply(lambda x: get_finish(x.Termination,x.Moves),axis=1)
    
    # se séparer des colonnes inutiles
    df_games = df_games.drop(['Site','Date','UTCDate','UTCTime', 'White','Black','Result','WhiteElo','BlackElo',
                             'WhiteRatingDiff','BlackRatingDiff','Variant','Event','Termination'], axis=1)    
    df_games = df_games.set_index('DateTime').sort_index()
    df_games['GameId'] = np.arange(df_games.shape[0])
    return df_games

new_games = transform_data(df_games)


In [41]:
new_games = new_games[['GameId','TimeControl','MyColor','OpponentName','MyElo','OpponentElo','EloDifference','ECO','Moves','MyRatingDiff','OpponentRatingDiff','Finish','MyResult','Win']]

In [42]:
new_games.iloc[0]

GameId                                                                0
TimeControl                                                       600+0
MyColor                                                           Black
OpponentName                                                    arilico
MyElo                                                              1500
OpponentElo                                                        1404
EloDifference                                                        96
ECO                                                                 C20
Moves                 1. e4 e5 2. Qf3 Nf6 3. Nc3 b6 4. Bc4 Bb7 5. Nd...
MyRatingDiff                                                       -230
OpponentRatingDiff                                                    9
Finish                                                          TimeOut
MyResult                                                         Defeat
Win                                                             

In [43]:
new_games.to_csv('CleanLichessData.csv')

Notre jeu de données contient désormais les informations utiles pour analyser nos parties. Il est conçu de sorte à être facilement compréhensible par un humain.                   
La prochaine étape est d'en faire une étude exploratoire pour en dégager des tendances.               
On va aussi numériser et normaliser toutes les caractéristiques utiles pour pouvoir lui appliquer des modèles de Machine Learning

# Fin 


**Fait par Youness Khanjar**